# TCAD file exploration

We have received files from a client.  They are ....

# Shorten files for browsing

To shorten the files for browsing we can run a short shell script. This opens the zip that was received, and truncates each file at 100 lines long.

```{bash, eval=F}
# rm -rf shortened_appraisal_files
unzip original_data/Appraisal_Roll_History_1990.zip -d shortened_appraisal_files
find shortened_appraisal_files -name "*.TXT" -exec sed -i.full 100q {} \;
find shortened_appraisal_files -name "*.TXT.full" -exec rm {} \;
zip -r shortened_appraisal_files.zip shortened_appraisal_files
```

We can now attempt to load a shortened file using pandas

In [1]:
import pandas as pd

df = pd.read_csv("shortened_appraisal_files/Appraisal_Roll_History_1990_A/TCBC_SUM_1990_JURIS.TXT", sep = "|")
df.head()

,0000000003,0000,1990,02,0.56950,CI,Unnamed: 6,275,0,2923,...,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,4098.00,0.00,0.00.1,12.23,11.11,23.34
0,3,0,1990,3,0.4090,CO,NaN,275,0,2923,...,NaN,NaN,NaN,NaN,4098.0,0.0,0.0,16.76,0.00,16.76
1,3,0,1990,4,0.0001,CR,NaN,275,0,2923,...,NaN,NaN,NaN,NaN,4098.0,0.0,0.0,0.00,0.00,0.00
2,3,0,1990,8,1.6410,SD,NaN,275,0,2923,...,NaN,NaN,Y,NaN,4098.0,0.0,0.0,50.24,17.01,67.25
3,7,0,1990,1,1.2660,SD,NaN,25500,0,35000,...,NaN,NaN,Y,NaN,78000.0,0.0,0.0,836.55,150.93,987.48
4,7,0,1990,2,0.5695,CI,NaN,25500,0,35000,...,NaN,NaN,NaN,NaN,78000.0,0.0,0.0,232.75,211.46,444.21


In [5]:
# extract zip folder into a new folder
import zipfile
import os

# zip_file_path = "shortened_appraisal_files.zip"
zip_file_path = "original_data/Appraisal_Roll_History_1990.zip"
extract_folder_path = "data"

# Create the extract folder if it doesn't exist
if not os.path.exists(extract_folder_path):
    os.makedirs(extract_folder_path)

# Open the zip file and extract its contents to the extract folder
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extract_folder_path)

Challenge now is to use the *.TDF files to create tables.  I can think of two approaches.

1. The TDF files are SQL, so if those are fed to duckdb they should be able to create tables into which the TXT pipe-separated CSV files can be read.  There may be issues with the datatypes not matching (which would require mapping the current datatype definitions to duckdb datatypes by changing the words used to give the datatype to the columns).

2. Take the column names out of the TDF files and add them as the column names while reading the relevant CSV files into duckdb.  This would use duckdb's auto understanding of the column datatypes (so it would run, but it might guess wrongly and truncate or change data).

I think we should explore step 1 first.

## Creating tables using the TDF files

We have TDF files scattered through the \_A and \_B folders.  I have created a schema (a namespace) for the files from \_A called "folder_A" and "folder_B". So there are tables named the same thing in each of the schemas.  You can reference the tables as folder_A.TCBC_SUM_1990_JURIS and folder_B.TCBC_SUM_1990_JURIS 

We can use python to read each TDF file separately, create the table and then try to load the matching TXT file.  A little guidance on how to process a directory structure of files using Path and glob here:
http://howisonlab.github.io/datawrangling/faq.html#get-data-from-filenames

In [22]:
import csv
from pathlib import Path
import duckdb

con = duckdb.connect('duckdb-file.db') #  string to persist to disk
cursor = con.cursor()

# file_directory = 'shortened_appraisal_files/'
file_directory = 'data/'
# limit_to_file = 'TCBC_SUM_1990_JURIS'
limit_to_file = '*' # all files

# create schemas
cursor.execute("CREATE SCHEMA IF NOT EXISTS folder_A_TCBC;")
cursor.execute("CREATE SCHEMA IF NOT EXISTS folder_A_TXBC;")
cursor.execute("CREATE SCHEMA IF NOT EXISTS folder_B_TCBC;")
cursor.execute("CREATE SCHEMA IF NOT EXISTS folder_B_TXBC;")
# delete schemas that created previously
# cursor.execute("DROP SCHEMA IF EXISTS folder_A CASCADE")
# cursor.execute("DROP SCHEMA IF EXISTS folder_B CASCADE")

for filename in Path(file_directory).rglob(limit_to_file + '.TDF'):
    print(filename.parts)
    if "_A" in filename.parts[1] and "TCBC_" in filename.parts[2]:
        schema = "folder_A_TCBC"
    elif "_A" in filename.parts[1] and "TXBC_" in filename.parts[2]:
        schema = "folder_A_TXBC"
    elif "_B" in filename.parts[1] and "TCBC_" in filename.parts[2]:
        schema = "folder_B_TCBC"
    elif "_B" in filename.parts[1] and "TXBC_" in filename.parts[2]:
        schema = "folder_B_TXBC"
    else:
        exit("can't set schema")
    
    table_name = schema + "." + Path(filename).stem # e.g., A_TCBC_SUM_1990_JURIS

    # read .TDF file into string
    create_table_sql = Path(filename).read_text()
    # Need to alter table name to read in both _A and _B files
    create_table_sql = create_table_sql.replace(Path(filename).stem, table_name)
    
    # Here we have the table creation code in a string, so we can
    # swap datatypes out.
    # tried SMALLDATETIME --> DATETIME but was still giving errors
    # will need to fix this later.
    create_table_sql = create_table_sql.replace("SMALLDATETIME", "TEXT")
    create_table_sql = create_table_sql.replace("CREATE TABLE", "CREATE TABLE IF NOT EXISTS")    
    create_table_sql = f"DROP TABLE IF EXISTS {table_name}; "+ create_table_sql
    

    # execute that SQL with duckdb, this should create the table
#     already created table so no need to run
    #print(create_table_sql)
    #exit(1)
    cursor.execute(create_table_sql) 

    # copy CSV into duckdb. CSV is the matching .TXT
    path_to_csvpipefile = Path(filename).with_suffix(".TXT")
    # duckdb copy documentation: https://duckdb.org/docs/sql/statements/copy.html
    query = f"COPY {table_name} FROM '{path_to_csvpipefile}' ( DELIMITER '|')"
    # print(query)
    cursor.execute(query)

('data', 'Appraisal_Roll_History_1990_A', 'TXBC_SUM_1990_SUSP.TDF')
('data', 'Appraisal_Roll_History_1990_A', 'TCBC_SUM_1990_JURIS.TDF')
('data', 'Appraisal_Roll_History_1990_A', 'TXBC_SUM_1990_JURIS.TDF')
('data', 'Appraisal_Roll_History_1990_A', 'TCBC_SUM_1990_CFOR.TDF')
('data', 'Appraisal_Roll_History_1990_A', 'TXBC_SUM_1990_JURIS_EXMP.TDF')
('data', 'Appraisal_Roll_History_1990_A', 'TXBC_SUM_1990_SUSP_INIT.TDF')
('data', 'Appraisal_Roll_History_1990_A', 'TCBC_SUM_1990_GRANT_EXMP.TDF')
('data', 'Appraisal_Roll_History_1990_A', 'TXBC_SUM_1990_CFOR.TDF')
('data', 'Appraisal_Roll_History_1990_A', 'TXBC_SUM_1990.TDF')
('data', 'Appraisal_Roll_History_1990_A', 'TCBC_SUM_1990_SUSP.TDF')
('data', 'Appraisal_Roll_History_1990_A', 'TCBC_SUM_1990_JURIS_EXMP.TDF')
('data', 'Appraisal_Roll_History_1990_A', 'TCBC_SUM_1990_LEGAL.TDF')
('data', 'Appraisal_Roll_History_1990_A', 'TCBC_SUM_1990.TDF')
('data', 'Appraisal_Roll_History_1990_A', 'TCBC_SUM_1990_SUSP_INIT.TDF')
('data', 'Appraisal_Roll_Hi

In [3]:
#set up sql for dbdocs
for filename in Path(file_directory).rglob(limit_to_file + '.TDF'):

    # SQL table code with commas
    dbdocs_create_table = create_table_sql

    # Remove commas before closing parentheses using regular expressions
    dbdocs_create_table = dbdocs_create_table.replace("),", ")")

    # Print the updated SQL table code
    print(dbdocs_create_table)


In [23]:
# setup from https://duckdb.org/docs/guides/python/jupyter.html
import duckdb
import pandas as pd
# No need to import duckdb_engine
#  jupysql will auto-detect the driver needed based on the connection string!

# Import jupysql Jupyter extension to create SQL cells
%load_ext sql
%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [24]:
%sql duckdb:///duckdb-file.db

In [14]:
%%sql
SHOW TABLES -- no schema name

,name
0,TCBC_SUM_1990_JURIS
1,TCBC_SUM_1990_JURIS


Hey, duckdb implements all the same information schema names as postgres, so one can use the same queries to find the tables with their schaema names.

In [25]:
%%sql
SELECT schemaname AS schema_name, tablename AS table_name
FROM pg_catalog.pg_tables
WHERE schemaname != 'pg_catalog'
AND schemaname != 'information_schema'
ORDER BY schemaname, tablename ASC;

,schema_name,table_name
0,folder_A_TCBC,TCBC_SUM_1990
1,folder_A_TCBC,TCBC_SUM_1990_CFOR
2,folder_A_TCBC,TCBC_SUM_1990_GRANT_EXMP
3,folder_A_TCBC,TCBC_SUM_1990_JURIS
4,folder_A_TCBC,TCBC_SUM_1990_JURIS_EXMP
5,folder_A_TCBC,TCBC_SUM_1990_LEGAL
6,folder_A_TCBC,TCBC_SUM_1990_SUSP
7,folder_A_TCBC,TCBC_SUM_1990_SUSP_INIT
8,folder_A_TXBC,TCBC_SUM_1990
9,folder_A_TXBC,TCBC_SUM_1990_CFOR


TCBC_SUM_1990_JURIS - Suppose total of 134933 rows, rows are adding up everytime rerun (fixed now)

JURIS probably means "jurisdiction" which means a legal area.  This makes sense because the columns are about tax rates (and metadata about tax status, like 'freeport').  So possibly this file is a list of jurisdictions to which a parcel can belong (and therefore holds the rates that would apply to the parcel?). It is surprising to have 134,933 different jurisdictions though!

In [26]:
%%sql
SELECT * FROM folder_A_TCBC.TCBC_SUM_1990_JURIS;

,AcctNum,SufxId,TaxYear,Juris,Rate,JurisType,JurisCED,MdseVal,FrptVal,FFEVal,...,ExmpStatFlag,JurisPctFlag,FreeportFlag,FreeportStatus,AssessVal,TaxFrzVal,TaxBeforeFrz,GenFundTax,SinkFundTax,TotTax
0,0000000003,0000,1990,02,0.56950,CI,None,275,0,2923,...,None,None,None,None,4098.00,0.00,0.00,12.23,11.11,23.34
1,0000000003,0000,1990,03,0.40900,CO,None,275,0,2923,...,None,None,None,None,4098.00,0.00,0.00,16.76,0.00,16.76
2,0000000003,0000,1990,04,0.00010,CR,None,275,0,2923,...,None,None,None,None,4098.00,0.00,0.00,0.00,0.00,0.00
3,0000000003,0000,1990,08,1.64100,SD,None,275,0,2923,...,None,None,Y,None,4098.00,0.00,0.00,50.24,17.01,67.25
4,0000000007,0000,1990,01,1.26600,SD,None,25500,0,35000,...,None,None,Y,None,78000.00,0.00,0.00,836.55,150.93,987.48
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
134928,0000061017,0000,1990,01,1.26600,SD,None,0,0,0,...,None,None,Y,None,100653.00,0.00,0.00,1079.51,194.76,1274.27
134929,0000061017,0000,1990,02,0.56950,CI,None,0,0,0,...,None,None,None,None,100653.00,0.00,0.00,300.35,272.87,573.22
134930,0000061017,0000,1990,03,0.40900,CO,None,0,0,0,...,None,None,None,None,100653.00,0.00,0.00,411.67,0.00,411.67
134931,0000061017,0000,1990,04,0.00010,CR,None,0,0,0,...,None,None,None,None,100653.00,0.00,0.00,0.10,0.00,0.10


The table without a suffix (TCBC_SUM_1990) has only 28,086 rows.  Perhaps these are accounts for individual tax payers, but individual tax payers can have multiple parcels.

In [27]:
%%sql
SELECT * FROM folder_A_TCBC.TCBC_SUM_1990

,AcctNum,SufxId,TaxYear,RunDate,KeyCode,LoanCo,LoanNum,ExmpCode,LocStreet,LocHouse,...,Zip5,Zip4,Zip2,MailCnt,MailAddr1,MailAddr2,MailAddr3,MailAddr4,MailAddr5,ComboRate
0,0000000003,0000,1990,1992-07-06,None,0,None,None,MO-PAC CI,001004,...,78767,0971,None,4,A & A REALTY TAX SERVICE,INC,P O BOX 971,AUSTIN TX 78767-0971,None,2.61950
1,0000000007,0000,1990,1992-07-06,None,0,None,None,5 ST E,002811,...,78744,None,None,4,A & J CARPET/JANITORIAL,SERVICE INC,4122 TODD LANE,AUSTIN TX 78744,None,2.29450
2,0000000014,0000,1990,1992-07-06,None,0,None,None,KENTSHIRE CI,000603,...,78704,5615,None,4,A A A COMMERCIAL,STRIPING,603 KENTSHIRE CIR #B,AUSTIN TX 78704-5615,None,2.29450
3,0000000015,0000,1990,1992-07-06,None,0,None,None,BEN WHITE BV E,004818,...,78759,None,None,4,A A A CONSTRUCTION,INSPECTIONS INC,8500 NORTH MOPAC #813,AUSTIN TX 78759,None,2.08850
4,0000000018,0000,1990,1992-07-06,None,0,None,None,BURNET RD,004402,...,78765,4674,None,4,A A A FILTER SERVICE,CORP,P O BOX 4674,AUSTIN TX 78765-4674,None,2.29450
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28081,0000060425,0000,1990,1992-07-06,None,0,None,None,RED RIVER ST,000912,...,78703,None,None,3,JOSEPH SALEM,1500 SCENIC DR #106,AUSTIN TX 78703,None,None,2.29450
28082,0000060456,0000,1990,1992-07-06,None,0,None,None,WILLIAM CANNON DR W,000414,...,78745,5664,None,3,FLOWERS BY HAND,414 W WILLIAM CANNON #8,AUSTIN TX 78745-5664,None,None,2.29450
28083,0000060832,0000,1990,1992-07-06,None,0,None,None,AMERICAN DR,003404,...,78645,6500,None,3,THE PRIME GROUP,3404 AMERICAN DR,LAGO VISTA TX 78645-6500,None,None,2.61850
28084,0000060999,0000,1990,1992-07-06,None,0,None,None,YAGER LN W,000615,...,78753,None,None,4,CONCRETE CORING CO INC,ATTN: MARTHA TURNER,615 YAGER LANE WEST,AUSTIN TX 78753,None,1.91400


In [ ]:
%%sql
SELECT * FROM folder_A_TCBC.TCBC_SUM_1990